## Demonstration of loading data, using no resolver

#### Add the path of modules of the project to sys

In [10]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

#### Import necessary libraries

In [11]:
import pathlib

from pepys_import.core.formats.repl_file import REPFile
from pepys_import.core.store.data_store import DataStore

#### Declaring global variables

In [12]:
RESOURCES_PATH = os.path.join('..', 'repl_files')

data_store = DataStore("", "", "", 0, "sqlite_datastore.db", db_type='sqlite')
data_store.initialise()
file_path = os.path.join(RESOURCES_PATH, "missing_platform.rep")
rep = REPFile(str(file_path))

In [13]:
with data_store.session_scope() as session:
    # TODO: fix the following line, it gives operational error
    datafile = session.add_to_datafile_from_rep(rep.get_data_file_name(), rep.get_data_file_type())
    for rep_line in rep.get_lines():
        platform = session.add_to_platforms_from_rep(rep_line.get_platform(),"Fisher", "UK", "Public")
        sensor = session.add_to_sensors_from_rep("GPS", platform)
        session.add_to_states_from_rep(rep_line.get_timestamp(), datafile, sensor, rep_line.get_latitude(), rep_line.get_longitude(), rep_line.get_heading(), rep_line.get_speed())